# Large_Melt_Map

Calculate SIR and GRD melt onset dates by year for selected subset areas.

Also calculates average MOD across years at each pixel.

Saves SIR or GRD MOD data together with geolocation information in pickle files

Makes maps of various annual and/or average results.

FIXME: 
The pixel overlays are simply a scatter plot with rectangular markers and are not using EASE-grid projection

FIXME:
Add a location for where to find the RGI overlays
and read/display the RGI shapefiles

## Load in all the modules needed

In [ ]:
%matplotlib inline
# check if a windows machine, it needs special attention
# this extra step will bypass an error from mpl_toolkits.basemap
import os
if os.name == 'nt':
    os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share")
    os.environ["GDAL_DATA"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share", "gdal")
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
from pathlib import Path
import re
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import glob
from pathlib import Path

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', None)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 10)

In [ ]:
# Define the local machine location of CETB data cubes
# This directory is expected to contain subdirectories in the following hierarchy
# that duplicates the hierarchy on the Google Shared Drive NSIDC-SD-CETB/v1/, 
# for example:
# dataDir/F13_SSMI/N/nc_cubes/cubes_<regionName>
#
# outDir is the location where pkl files with the MOD data frames and geotiffs
# will be stored. We are keeping it separate from the large cubefile data store
# so that individual people can keep track of their own output files separately
user = 'Joan' #Mariah #MJWindows #MJMac
if ('Joan' == user):
    #dataDir = '/mnt/data3/cetb/nsidc0630_v1/' #jmr machine fringe 
    dataDir = Path(Path.home(), 'rdrive', 'jmr204group','CETB_cubes')
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
#    outDir = Path(Path.home(), 'rdrive', 'jmr204group', 'ramage', 'geotiffs')
    outDir = Path(Path.home(), 'noisy')
elif ('Mariah' == user):
    dataDir = Path('R:\\jmr204group\CETB_cubes')  # Mariah's PC
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'ipynb_melt_onset_plots') # may want to update this to a geotiff output directory
elif ('MJWindows' == user):
    dataDir = Path('Z:/mj On My Mac/nsidc0630_v1') # Mary Jo's Windows machine
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'ipynb_melt_onset_plots') # may want to update this to a geotiff output directory
elif ('MJMac' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mary Jo's Mac
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')  
    outDir = Path(Path.home(), 'nsidc0630_v1')
else:
    raise ValueError("unknown user= %s\n" % (user) )
    
%cd $scriptDir
outDir, dataDir, user

In [ ]:
# load the custom functions
from CETB_IO import read_Tb_whole
from CETB_IO import coords
from CETB_algorithms import calc_DAV
from CETB_IO import find_cube_offset
from CETB_IO import grid_locations_of_subset
from CETB_IO import years_for
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array
from CETB_algorithms import end_high_DAV
from CETB_IO import read_Tb_std_dev
from CETB_IO import read_Tb
from CETB_IO import write_MOD_df_to_geotiff

## Specify inputs

In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='AKYukon'  #'GLaIL'  #make this the same syntax as cubefilenames and sub-directory
sat_GRD='F18'  #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sat_SIR= 'F18'
sensor_GRD='SSMIS'#'AMSRE', 'SSMI', etc.
sensor_SIR='SSMIS'
channel_GRD='37V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
channel_SIR='37V'
alg_GRD='GRD'   #SIR or GRD
alg_SIR='SIR'

# set the sir to grd factor, depends on the channel
if (re.match('^[389]', channel_GRD)):
    sir_2_grd_factor = 8 # assume 3.125 km to 25 km
elif (re.match('^[12]', channel_GRD)):
    sir_2_grd_factor = 4 # assume 6.25 km to 25 km
else:
    raise ValueError("Cannot determine sir_2_grd_factor from channel %s\n" % (channel_GRD) )

cubeType_GRD = channel_GRD + '-' + alg_GRD
cubeType_SIR = channel_SIR + '-' + alg_SIR
  
if ('SSMI' == sensor_GRD) or ('SSMIS' == sensor_GRD):
    provider='CSU' 
    version='v1.*'
elif 'AMSRE' == sensor_GRD:
    provider='RSS'
    version='v1.3'

hemName = 'N'    

# on Joan's machine
#datadir_GRD = dataDir + sat_GRD+'_'+sensor_GRD+'/'+region+'/' 
#datadir_SIR = dataDir + sat_SIR+'_'+sensor_SIR+'/'+region+'/' 
# on MJ's machine
datadir_GRD = "%s/%s_%s/%s/nc_cubes/cubes_%s/" % (
    dataDir, sat_GRD, sensor_GRD, hemName, region )
datadir_SIR = "%s/%s_%s/%s/nc_cubes/cubes_%s/" % (
    dataDir, sat_SIR, sensor_SIR, hemName, region )

# prefix filepath
prefix_GRD = 'CETB.cubefile.'+region+'.'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'-'+provider+'-'+version
prefix_SIR = 'CETB.cubefile.'+region+'.'+sat_SIR+'_'+sensor_SIR+'-'+channel_SIR+'-'+alg_SIR+'-'+provider+'-'+version

Years=years_for(sat_GRD)
#might want to truncate Years to subset if very slow during testing
#if we give it more years than available what do we want it to do? 
#warn me but return what it finds

In [ ]:
# SPECIFY latitude and longitude in decimal degrees, need to choose lat/lon corners so that we will load
# in a rectangle of pixels within the corners of these coordinates
areaname='fairbanks' #'gsl' #'hunza' #'vatna' etc

if ('vatna' == areaname):
    lat_start=63.75  
    lat_end=64.88    
    lon_start=-20 
    lon_end=-15  
    #Enter a site name for titles of plots
    Site='Vatnajokull, Iceland'
elif 'hunza' == areaname:
    lat_start=35.9  
    lat_end=37.1   
    lon_start=74 
    lon_end=76 
    #Enter a site name for titles of plots
    Site='Hunza Basin'
elif 'gsl' == areaname:
    lat_start=59.00  
    lat_end=67.00   
    lon_start=-119.00 
    lon_end=-107.00
    #Enter a site name for titles of plots
    Site='Great Slave Lake, Canada'
elif 'bathurst_range' == areaname:
    lat_start=60.00  
    lat_end=67.25   
    lon_start=-119.00 
    lon_end=-107.50
    #Enter a site name for titles of plots
    Site='Bathurst Caribou Range, NWT'
elif 'bathurst_range2' == areaname:
    lat_start=63.00  
    lat_end=65.500   
    lon_start=-117.500 
    lon_end=-112.00
    #Enter a site name for titles of plots
    Site='Bathurst Caribou Range subset, NWT'
elif ('barrow' == areaname):
    lat_start=69.50  
    lat_end=71.50    
    lon_start=-158 
    lon_end=-152  
    #Enter a site name for titles of plots
    Site='Barrow/Utkiagvik, AK'  
elif ('fairbanks' == areaname):
    lat_start=63.0  
    lat_end=66.7    
    lon_start=-151.8
    lon_end=-143.4  
    #Enter a site name for titles of plots
    Site='Fairbanks, AK'
else: 
    raise ValueError("Unknown area name=%s" % (areaname)) 

In [ ]:
# get cube offset for finding row/col
# function is region specific
find_cube_offset(region, cubeDir=datadir_SIR, cubeType=cubeType_SIR, verbose=False)

In [ ]:
# get the GRD pixel IDs for the lat/lon rectangle chosen
# and then calculate the corrsponding SIR pixel row/col numbers
rows_cols_GRD=coords(datadir_GRD, prefix_GRD, lat_start, lat_end, lon_start, lon_end)
rows_cols_env = tuple(np.array(rows_cols_GRD) * sir_2_grd_factor)
print(rows_cols_GRD)
print(rows_cols_env)

In [ ]:
# load in SIR TB data
# Truncate Years here for speed 
#subYears = Years[0:2]
subYears = Years

data_SIR = read_Tb_whole(datadir_SIR, prefix_SIR, subYears,
                        rows_cols_env[0], rows_cols_env[1], rows_cols_env[2], rows_cols_env[3])

# Information passed back from "read_Tb_whole" reader includes:
CETB_SIR = data_SIR['TB']   # 3-D Tb time-series array of TB
cal_date = data_SIR['cal_date'] # 1-D array of dates, these will get passed to later functions
cal_year = data_SIR['cal_year']    # 1-D array of years
cal_month = data_SIR['cal_month']   # 1-D array of months
# data_SIR['latitude'], data_SIR['longitude'] # 2-D arrays of subset pixel lat/lons
# data_SIR['x'], data_SIR['y'] # 2-D arrays of subset pixel projected x/y
# data_SIR['gpd'] # name of EASE2 projection that the subset was derived from

# load GRD Tb data
data_GRD = read_Tb_whole(datadir_GRD, prefix_GRD, subYears,
                         rows_cols_GRD[0], rows_cols_GRD[1], rows_cols_GRD[2], rows_cols_GRD[3])

CETB_GRD = data_GRD['TB']

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_SIR = calc_DAV(data_SIR['TB'])
DAV_GRD = calc_DAV(data_GRD['TB'])

In [ ]:
DAV_GRD.shape, data_GRD['TB'].shape, DAV_SIR.shape

# generate histogram - SIR - all data
# This histogram will include all points that are in the CETB_SIR array
``` 
year=2007
Tb_threshold=240  #set above by sensor
data = CETB_SIR[cal_year==year]
#data = CETB_SIR[:] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.hist(data, bins)
ax.set_title('All data SIR Histogram'+' '+Site)
ax.axvline(x=Tb_threshold, color='red')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)') 
```

## create MOD in data frames with geolocation

In [ ]:
# get array of average MODs for SIR pixels for all the years loaded
#Colorado (Johnson et al 2020) used 5 times in 7 day window, for locations in Colorado
#Patagonia (Monahan and Ramage 2010) and Yukon (Semmens et al 2013?) used 3 times in 5 day window

# From Matias Fall 2022 forward facing indexer to assign rolling sum value to the beginning of window
# Setting the window_size sets the number of observations,'14' would be 7 days (2 measurements per day)
# If you don't want to use the forward facing indexer, then change "window" to a numeral, this will assign
# the rolling sum value to the end of the window
MOD_window = 10
MOD_count = 3
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=MOD_window)
window = indexer   

# number of Tb/DAV exceedances to trigger MOD
#From Johnson et al 2020 AMSRE rSIR Tb >= 249 DAV>=13 and AMSRE GRD Tb>=243 DAV>=14
#From Johnson et al 2020 SSMI rSIR and GRD Tb>=247 DAV>=10
DAV_threshold = 10
Tb_threshold = 247

# Number of Tb/DAV exceedances to trigger EHD = end of high DAV
# At the current time, EHD is not using the forward facing indexer, using default behavior
# From Matias Fall 2022
EHD_window = 20 
EHD_count = 7 

In [ ]:
# sir MOD array - MOD will be in day of year (DOY) #changeback to Years for all years
MOD_DOY_df, meltflag_df, EHD_DOY_df, EHDflag_df = MOD_array(
    datadir_SIR, prefix_SIR, data_SIR, DAV_SIR, rows_cols_env, 
    subYears, window, MOD_count, EHD_window, EHD_count, DAV_threshold, Tb_threshold)

In [ ]:
MOD_DOY_df, EHD_DOY_df

In [ ]:
print(meltflag_df)

In [ ]:
print(EHDflag_df)

In [ ]:
## MOD of the GRD pixel - avg all years
MOD_DOY_GRD_df, meltflag_GRD_df, EHD_DOY_GRD_df, EHDflag_GRD_df = MOD_array(
    datadir_GRD, prefix_GRD, data_GRD, DAV_GRD, rows_cols_GRD, 
    subYears, window, MOD_count, EHD_window, EHD_count, DAV_threshold, Tb_threshold)

In [ ]:
MOD_DOY_GRD_df 

In [ ]:
EHD_DOY_GRD_df

In [ ]:
print(meltflag_GRD_df)

In [ ]:
print(EHDflag_GRD_df)

# Notes about changes from original notebooks:

The old notebooks used to call MOD_array to get the average MOD for a set of years 
and then call MOD_array_year for a given year of interest.

Now we just call MOD_array for SIR data and GRD data, and get back a data frame with 
MOD columns for each individual year, and one column for the avg MOD for all the years.

This will run much faster, and can be saved and just re-read from a saved file on disk.

N.B. With this approach, we can create pickle files once, and then 
read saved pickle files with this command:

new = pd.read_pickle(MOD_DOY_filename)

# Save the MOD by year data frames for SIR and GRD to pickle files (with lat/lon geolocation)

Also saving geolocation and melt onset flag data frames

In [ ]:
subYears

In [ ]:
modsDir = "%s/MODs" % outDir
if not os.path.isdir(modsDir):
    os.makedirs(modsDir)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.MOD.%1dof%02d.pkl" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, 
    subYears[0], subYears[-1], MOD_count, MOD_window)
MOD_DOY_df.to_pickle(filename)
print("MOD_DOY dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.MOD.%1dof%02d.pkl" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, 
    subYears[0], subYears[-1], MOD_count, MOD_window)
MOD_DOY_GRD_df.to_pickle(filename)
print("MOD_DOY_GRD dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.meltflag.%1dof%02d.pkl" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, 
    subYears[0], subYears[-1], MOD_count, MOD_window)
meltflag_df.to_pickle(filename)
print("meltflag_df dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.meltflag.%1dof%02d.pkl" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, 
    subYears[0], subYears[-1], MOD_count, MOD_window)
meltflag_GRD_df.to_pickle(filename)
print("meltflag_GRD_df dataframe saved to %s\n" % filename)

# Save the MOD by year as geotiff images

This will save one geotiff for each year plus one each for the Avg, Min and Max MODs

In [ ]:
outbasename = "%s/%s-%s-%s.%s.%s.%s.MOD.%1dof%02d" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, MOD_count, MOD_window)
out = write_MOD_df_to_geotiff(MOD_DOY_df, data_SIR['gpd'], outbasename, verbose=True)

outbasename = "%s/%s-%s-%s.%s.%s.%s.MOD.%1dof%02d" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, MOD_count, MOD_window)
out = write_MOD_df_to_geotiff(MOD_DOY_GRD_df, data_GRD['gpd'], outbasename, verbose=True)

# Save the EHD by year data frames for SIR and GRD to pickle files (with lat/lon geolocation)

Also saving geolocation and end of high DAV flag data frames

In [ ]:
modsDir = "%s/MODs" % outDir
if not os.path.isdir(modsDir):
    os.makedirs(modsDir)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.EHD.%1dof%02d.pkl" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, 
    subYears[0], subYears[-1], EHD_count, EHD_window)

EHD_DOY_df.to_pickle(filename)
print("EHD_DOY dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.EHD.%1dof%02d.pkl" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, 
    subYears[0], subYears[-1], EHD_count, EHD_window)
EHD_DOY_GRD_df.to_pickle(filename)
print("EHD_DOY_GRD dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.EHDflag.%1dof%02d.pkl" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, 
    subYears[0], subYears[-1], EHD_count, EHD_window)
EHDflag_df.to_pickle(filename)
print("meltflag_df dataframe saved to %s\n" % filename)

filename = "%s/%s-%s-%s.%s.%s.%s.%s-%s.EHDflag.%1dof%02d.pkl" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, 
    subYears[0], subYears[-1], EHD_count, EHD_window)
EHDflag_GRD_df.to_pickle(filename)
print("EHDflag_GRD_df dataframe saved to %s\n" % filename)

# Save the EHD by year as geotiff images

This will save one geotiff for each year plus one each for the Avg, Min and Max EHDs

In [ ]:
outbasename = "%s/%s-%s-%s.%s.%s.%s.EHD.%1dof%02d" % (
    modsDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, alg_SIR, EHD_count, EHD_window)
out = write_MOD_df_to_geotiff(EHD_DOY_df, data_SIR['gpd'], outbasename, verbose=True)

outbasename = "%s/%s-%s-%s.%s.%s.%s.EHD.%1dof%02d" % (
    modsDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, alg_GRD, EHD_count, EHD_window)
out = write_MOD_df_to_geotiff(EHD_DOY_GRD_df, data_GRD['gpd'], outbasename, verbose=True)

## Display maps of the MOD data

This next example displays the requested SIR MOD map

#how to create subplots with basemap (w maps) https://basemaptutorial.readthedocs.io/en/latest/subplots.html

In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - SIR
# Choose plot type - 'year' for one year of interest, and specify the year,
# or 'all' for the average of all years
plot_type='year'  
year=2010
#plot_type='all'

if plot_type == 'all':
    array = MOD_DOY_df['Avg']
    label = 'AvgDOY'+str(subYears[0])+'-'+str(subYears[-1])
    title = sensor_SIR + '-' + sat_SIR + '-' + Site + ' - MOD - (Avg DOY '+str(subYears[0])+'-'+str(subYears[-1])+')'
elif plot_type == 'year':
    array = MOD_DOY_df[year]
    label = str(year)+'DOY' 
    title = sensor_SIR + '-' +sat_SIR + '-'+ Site + ' - MOD - (' + str(year) + ' DOY)'

# Set a few common things
graticule_fontsize = 4

# create figure and axes instances
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',
            lon_0=(lon_start+lon_end)/2,
            lat_0=(lat_start+lat_end)/2,
            lat_ts=90.,
            llcrnrlat=(lat_start-.3),
            urcrnrlat=(lat_end+.5),
            llcrnrlon=(lon_start-.3),
            urcrnrlon=(lon_end+.3),
            rsphere=6371200.,
            resolution='l',
            area_thresh=10000, 
            epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels, labels=[1,0,0,0], fontsize=graticule_fontsize)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians, labels=[0,0,0,1], fontsize=graticule_fontsize)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#specific shapefile overlays can be added here:
#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
#m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Iceland', color='red', linewidth=1)

x,y = m(MOD_DOY_df['longitude'].values, MOD_DOY_df['latitude'].values)
# scatter plot size argument is size in points**2, default
# is rcParams['lines.markersize'] ** 2.
scatter_size = 24.
im = m.scatter(x, y, c=array, s=scatter_size, marker='s',
              lw=0, cmap='BuPu_r', alpha=.6) 
plt.title(title)

# Get the current axes and configure placement so colorbar will
# be 5% of ax width and the padding will be 0.05 in.
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax, label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()

filename = "%s/%s.%s.%s.%s.%s.%s.png" % (
    modsDir, region, areaname, sat_SIR, channel_SIR, alg_SIR, label)
plt.savefig(filename)
print("image saved to %s\n" % filename)

In [ ]:
#color bars here https://matplotlib.org/stable/tutorials/colors/colormaps.html
#documentation on basemap https://matplotlib.org/basemap/users/laea.html

## Map of GRD (left) and SIR (right) MOD maps

In [ ]:
# Trial-and-error decide on size for GRD vs. SIR scatter markers
# scatter size is in points**2, with default of
# rcParams['lines.markersize'] ** 2.
#np.sqrt((1200)/8)^2
sGRD=700
sSIR=12

In [ ]:
# 2plots left (GRD) and right (SIR)
#redo basemap to be in easegrid projection EASE2.0
#
# Choose plot type - 'year' for one year of interest, and specify the year,
# or 'all' for the average of all years
plot_type='year'  
year=2019
#plot_type='all'


if plot_type == 'all':
    label = 'MOD.Avg.%1dof%02d.%4d-%4d' % (MOD_count, MOD_window, subYears[0], subYears[-1])
    sir_array = MOD_DOY_df['Avg']
    sir_title = sensor_SIR + '-' + sat_SIR + '-' + Site + ' rSIR MOD (Avg DOY '+str(subYears[0])+'-'+str(subYears[-1])+')'
    grd_array = MOD_DOY_GRD_df['Avg']
    grd_title = sensor_GRD + '-' + sat_GRD + '-' + Site + ' GRD MOD (Avg DOY '+str(subYears[0])+'-'+str(subYears[-1])+')'
elif plot_type == 'year':
    label = 'MOD.%1dof%02d.%4d' % (MOD_count, MOD_window, year)
    sir_array = MOD_DOY_df[year]
    sir_title = sensor_SIR + '-' + sat_SIR + '-' + Site + ' rSIR MOD (' + str(year) + ' DOY)'
    grd_array = MOD_DOY_GRD_df[year]
    grd_title = sensor_GRD + '-' + sat_GRD + '-' + Site + ' GRD MOD (' + str(year) + ' DOY)'

fig, axes = plt.subplots(1, 2, figsize=(14,5))

# Set a few common things
graticule_fontsize = 7

#min day of year to plot
minday=75
#max day of year to plot
maxday=175

# create polar stereographic Basemap instance.
axes[0].set_title(grd_title)
m = Basemap(ax=axes[0],projection='stere',
            lon_0=(lon_start+lon_end)/2,
            lat_0=(lat_start+lat_end)/2,lat_ts=90.,
            llcrnrlat=(lat_start-.3),
            urcrnrlat=(lat_end+.5),
            llcrnrlon=(lon_start-.3),
            urcrnrlon=(lon_end+.3),
            rsphere=6371200.,
            resolution='l',
            area_thresh=10000, 
            epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,1.0)
m.drawparallels(parallels, labels=[1,0,0,0], fontsize=graticule_fontsize)
# draw meridians
meridians = np.arange(0,360.,1.0)
m.drawmeridians(meridians, labels=[0,0,0,1], fontsize=graticule_fontsize)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
#m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

x,y = m(MOD_DOY_GRD_df['longitude'].values, MOD_DOY_GRD_df['latitude'].values)
im0 = m.scatter(x, y, c=grd_array, 
                s=sGRD, marker='s',lw=0,cmap='BuPu_r', alpha=.6, vmin=minday, vmax=maxday) 

# Get the current axes and configure placement so colorbar will
# be 5% of ax width and the padding will be 0.08 in.
divider = make_axes_locatable(axes[0])
cax = divider.append_axes("right", size="5%", pad=0.08)
cbar = plt.colorbar(im0, cax=cax, label='DOY')

axes[1].set_title(sir_title)
m = Basemap(ax=axes[1],projection='stere',
            lon_0=(lon_start+lon_end)/2,
            lat_0=(lat_start+lat_end)/2,lat_ts=90.,
            llcrnrlat=(lat_start-.3),
            urcrnrlat=(lat_end+.5),
            llcrnrlon=(lon_start-.3),
            urcrnrlon=(lon_end+.3),
            rsphere=6371200.,
            resolution='l',
            area_thresh=10000, 
            epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,1.0)
m.drawparallels(parallels, labels=[1,0,0,0], fontsize=graticule_fontsize)
# draw meridians
meridians = np.arange(0,360.,1.0)
m.drawmeridians(meridians, labels=[0,0,0,1], fontsize=graticule_fontsize)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#m.readshapefile('/home/mij216/ExploringCETB/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)
#m.readshapefile('/mnt/data3/rgi6.0/06_rgi60_Iceland', 'Vatnajokull', color='red', linewidth=1)

x,y = m(MOD_DOY_df['longitude'].values, MOD_DOY_df['latitude'].values)
im1 = m.scatter(x, y, c=sir_array, 
                s=sSIR, marker='s', lw=0,cmap='BuPu_r', alpha=.6,vmin=minday,vmax=maxday) 
#fig.colorbar(im1, ax=axes[1],label='DOY')
#plt.clim(75,175)  #color bar limits

# Get the current axes and configure placement so colorbar will
# be 5% of ax width and the padding will be 0.05 in.
divider = make_axes_locatable(axes[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im1, cax=cax, label='DOY')

plt.tight_layout(pad=3.0)


filename = "%s/%s.%s.%s.%s.%s.%s.png" % (
    modsDir, region, areaname, sat_SIR, channel_SIR, "GRDvsSIR", label)
plt.savefig(filename)
print("image saved to %s\n" % filename)

# Apparently order is important here: you have to save the figure before you do plt.show()
plt.show()

In [ ]:
#setup a dataframe so that all the data from ssmi and ssmis are read into one array for long term average
#need to decide which sensor has priority when there is overlap
#can create a reader that loops through enough sensors to get a full time series 
#(skip F10- orbit very elliptical and F19 - short)
#create that time series for the analysis, save for analysis
#
#save dataframes with MOD so that we can do other analysys
#pickle the data frame that is inside the MOD_array_DOY function 
#
#check in CETB_analysis.py that when it does the count/window it is in fact 
#saving the first occurrence of the count, even though it doesn't know until it gets to 3x

In [ ]:
#Add to this
#Plot of which pixels have melted as of a specific day

In [ ]:
#figure out refreeze? then from MOD to Freeze, count # of dates with Tb>=threshold (with or without DAV? might depend on site)

In [ ]:
#Add to this
#Plot of how many days (or occurrences) experienced melt 


In [ ]:
#Add to this
#Plot of how many short (1-2 occurrences) melt days EMEs prior to the first MOD
#From Jan 1 - MOD, Number of days with Tb>= threshold and DAV>= threshold (same thresholds)